# Mapboxgl Python Library for location data visualizaiton

https://github.com/mapbox/mapboxgl-jupyter


# Legend Styles and Controls

In [1]:
import pandas as pd
import os
from mapboxgl.utils import *
from mapboxgl.viz import *


# Set Mapbox Acces Token; Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Create a visualization from example data file

In [25]:
# Generate data breaks using numpy quantiles and color stops from colorBrewer
measure = 'Avg Medicare Payments'
color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(1,9)]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

# Create the viz from the dataframe
viz = CircleViz('../data/healthcare_points.geojson',
                access_token=token, 
                color_property='Avg Medicare Payments',
                color_stops=color_stops,
                radius=2.5,
                stroke_color='black',
                stroke_width=0.2,
                center=(-95, 40),
                zoom=3,
                below_layer='waterway-label')

# Show the viz
viz.show()

## Change the viz legend colors

In [32]:
viz.legend_fill = '#f0f0ef'
viz.legend_text_color = '#000000'

# header fill only available for vertical layout
viz.legend_layout = 'vertical'
viz.legend_header_fill = '#819092'

viz.show()

## Change the viz legend layout

In [33]:
viz.legend_layout = 'horizontal'
viz.show()

## Create a graduated cricle viz based on two data properties

In [ ]:
# Generate data breaks and color stops from colorBrewer
measure_color = 'Avg Covered Charges'
color_breaks = [round(df[measure_color].quantile(q=x*0.1), 2) for x in range(2, 10)]
color_stops = create_color_stops(color_breaks, colors='Blues')

# Generate radius breaks from data domain and circle-radius range
measure_radius = 'Avg Medicare Payments'
radius_breaks = [round(df[measure_radius].quantile(q=x*0.1), 2) for x in range(2,10)]
radius_stops = create_radius_stops(radius_breaks, 0.5, 10)

# Create the viz
viz2 = GraduatedCircleViz('../data/healthcare_points.geojson', 
                          access_token=token,
                          color_property = "Avg Covered Charges",
                          color_stops = color_stops,
                          radius_property = "Avg Medicare Payments",
                          radius_stops = radius_stops,
                          stroke_color = 'black',
                          stroke_width = 0.5,
                          center = (-95, 40),
                          zoom = 3,
                          opacity=0.75,
                          below_layer = 'waterway-label')

viz2.show()

## Create a clustered circle map

In [ ]:
#Create a clustered circle map
color_stops = create_color_stops([1, 10, 25, 50, 75, 100], colors='YlOrBr')

viz4 = ClusteredCircleViz('../data/healthcare_points.geojson', 
                  access_token=token,
                  color_stops = color_stops,
                  stroke_color = 'black',
                  radius_stops = [[1,5], [10, 10], [50, 15], [100, 20]],
                  radius_default = 2,
                  cluster_maxzoom = 10,
                  cluster_radius = 30,
                  label_size = 12,
                  opacity = 0.9,
                  center = (-95, 40),
                  zoom = 3
                 )

viz4.show()